<a href="https://colab.research.google.com/github/JairEsc/Mat_Apl/blob/main/Opt_T9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

FR: $\beta_{k+1}^{FR}=\frac{||\nabla f(x_{k+1})||^2}{||\nabla f(x_{k})||^2}$

PR: $\beta_{k+1}^{PR}=\frac{\nabla^T f(x_{k+1})\cdot( \nabla f(x_{k+1})-\nabla f(x_k))}{||\nabla f(x_{k})||^2}$

FR-PR: $\beta_{k+1}^{FR-PR}=\left\{
	\begin{array}{ll}
		-\beta^{FR}_{k+1}, \text{ if } \beta_{k+1}^{PR}\leq -\beta_{k+1}^{FR}\\
		\beta^{PR}_{k+1}, \text{ if } |\beta_{k+1}^{PR}|\leq \beta_{k+1}^{FR}\\
		\beta^{FR}_{k+1}, \text{ if } \beta_{k+1}^{PR}> \beta_{k+1}^{FR}\\
	\end{array}
\right.$

HS $\beta_{k+1}^{HS}=\frac{\nabla^T f(x_{k+1})\cdot( \nabla f(x_{k+1})-\nabla f(x_k)}{ (\nabla f(x_{k+1})-\nabla f(x_k))^T\cdot d_k}$

General
Algorithm:
$$
d_0=-\nabla f(x_0), k=0$$
$$
while(||d_k||>tol_g):$$
$$\text{Compute } \alpha_k \text{ using line search algorithm}$$
$$\text{Update } x_{k-1}=x_k+\alpha_kd_k$$
$$\text{Compute } \nabla f(x_{k+1})$$
$$\text{Choose } \beta_{k+1} \text{ using a method.}$$
$$\text{Update } d_{k+1}=-\nabla f(x_{k+1})+\beta_{k}\cdot d_k$$
$$k=k+1$$


In [2]:
from scipy import optimize
import numpy as np

In [44]:
#Gradiente conjugado con varios metodos---------------------------------
def choose_beta(grad_f_kp1,grad_f_k,d_k,method):
    if (method=='FR'):
        return (np.dot(grad_f_kp1,grad_f_kp1-np.array(grad_f_kp1)))/(np.dot(grad_f_k,grad_f_k))
    if (method=='PR'):
        return (np.dot(grad_f_kp1,grad_f_kp1))/(np.dot(grad_f_k,grad_f_k))
    if (method=='HS'):
        return (np.dot(grad_f_kp1,grad_f_kp1-np.array(grad_f_kp1)))/(np.dot(grad_f_kp1-grad_f_k,d_k))
    if (method=='FR-PR'):
        bFR=(np.dot(grad_f_kp1,grad_f_kp1-np.array(grad_f_kp1)))/(np.dot(grad_f_k,grad_f_k))
        bPR=(np.dot(grad_f_kp1,grad_f_kp1))/(np.dot(grad_f_k,grad_f_k))
        if(abs(bPR)<=bFR):
            return bPR
        else:
            if(bPR<-bFR):
                return -bFR
            else:
                return bFR

def GC(f,grad_f,x_0,tol_g=10**(-4),max_iter=10**3,method='FR'):
    d_0=-1.*np.array(grad_f(x_0))
    g_k=-d_0
    k=0
    while(np.linalg.norm(d_0)>=tol_g and k<max_iter):
        alpha_k_gc=optimize.line_search(f=f,myfprime=grad_f,xk=x_0,pk=d_0,c1=0.001,c2=0.9)[0] #Tam. de paso
        x_1=x_0+alpha_k_gc*d_0 #update
        g_km1=grad_f(x_1) 
        beta_k_gc=choose_beta(grad_f_kp1=g_km1,grad_f_k=g_k,d_k=d_0,method=method)
        d_0=-np.array(g_km1)+beta_k_gc*d_0
        x_0=x_1
        k+=1
    print("iter",k)
    return x_0
#-----------------------------------

In [ ]:
lamb=100
G=[]
mu=0.01
def fun_obj(X):#X es una matriz de nxn
    f=0
    for i in range(X.shape(0)):
        for j in range(X.shape(1)):
            f+=(X[i,j]-G[i,j])^2+lamb*(np.sqrt((X[i,j]-X[i+1,j])**2+mu)+np.sqrt((X[i,j]-X[i-1,j])**2+mu)+np.sqrt((X[i,j]-X[i,j+1])**2+mu)+np.sqrt((X[i,j]-X[i,j-1])**2+mu))
    return f